# Calculation of AC sensation levels

18-01-2021 GC

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
def read_AC_thresholds():
    # function to read threshold in dB SPL op de eardrum
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/AC_thr_third.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name='AC_thr_eardrum', header=0, nrows=85)
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.fillna(pd.NA)
    return df

In [3]:
def read_ISTS_SPL():
    # function to read ISTS dB SPL for 1/3 thirdbands, 65 dB & 55 dB
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/constants.xlsx'
    p_in = Path(f_in)   
    col_to_use = list(range(20))
    df = pd.read_excel(p_in, sheet_name='ISTS_sound_pressure', 
                       header=0, nrows=2, usecols=col_to_use)
    df = df.fillna(pd.NA)
    df = df.rename(columns={'Unnamed: 0' : 'Signal'})
    df = df.set_index(['Signal'])
    s65 = pd.Series(df.iloc[0])
    s55 = pd.Series(df.iloc[1])
    return (s65, s55)

In [4]:
def read_HRTF():
    # function to read HRTF data for the AC path, collected by Stenfelt
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/constants.xlsx'
    p_in = Path(f_in)   
    col_to_use = list(range(20))
    df = pd.read_excel(p_in, sheet_name='HRTF_KEMAR_Stenfelt', 
                       header=0, nrows=3, usecols=col_to_use)
    df = df.fillna(pd.NA)
    df = df.rename(columns={'Unnamed: 0' : 'Angle'})
    df = df.set_index(['Angle'])
    df = df.round(1)
    s0 = pd.Series(df.iloc[0])
    s90 = pd.Series(df.iloc[1])
    s270 = pd.Series(df.iloc[2])
    return (s0, s90, s270)

In [5]:
def ISTS_HRTF_65():
    # ISTS 65 dB corrected with HRTF for 0, 90 and 270 degrees, return list of 3 pd.Series
    ists65 = read_ISTS_SPL()[0]
    hrtf = read_HRTF()
    res = list()
    for s in hrtf:
        corr = s + ists65
        res.append(corr)
    return res

In [6]:
def ISTS_HRTF_55():
    # ISTS 55 dB corrected with HRTF for 0, 90 and 270 degrees, return list of 3 pd.Series
    ists55 = read_ISTS_SPL()[1]
    hrtf = read_HRTF()
    res = list()
    for s in hrtf:
        corr = s + ists55
        res.append(corr)
    return res

In [7]:
def sl_65():
    # return list with AC sensation levels  for 0, 90 and 270 degrees
    act = read_AC_thresholds()
    info = act[['Study_ID', 'Device']]
    act1 = act.drop(columns=['Study_ID', 'Device'])
    l = list()
    for i in range(len(ISTS_HRTF_65())):
        sl = ISTS_HRTF_65()[i] - act1
        sl = pd.concat([info, sl], axis=1) 
        l.append(sl)
    return l

In [8]:
def sl_55():
    # return list with AC sensation levels  for 0, 90 and 270 degrees
    act = read_AC_thresholds()
    info = act[['Study_ID', 'Device']]
    act1 = act.drop(columns=['Study_ID', 'Device'])
    l = list()
    for i in range(len(ISTS_HRTF_55())):
        sl = ISTS_HRTF_55()[i] - act1
        sl = pd.concat([info, sl], axis=1) 
        l.append(sl)
    return l

In [9]:
sl65 = sl_65()
sl55 = sl_55()

In [10]:
# write results to xlsx file in Master directory
fout = '/media/guido/LACIE/Cingle_Guido/Master/SL_AC.xlsx'
pout = Path(fout)
with pd.ExcelWriter(pout) as writer:
    sl65[0].to_excel(writer, sheet_name='AC_SL_65dB_0deg')
    sl65[1].to_excel(writer, sheet_name='AC_SL_65dB_90deg')
    sl65[2].to_excel(writer, sheet_name='AC_SL_65dB_270deg')
    sl55[0].to_excel(writer, sheet_name='AC_SL_55dB_0deg')
    sl55[1].to_excel(writer, sheet_name='AC_SL_55dB_90deg')
    sl55[2].to_excel(writer, sheet_name='AC_SL_55dB_270deg')